In [1]:
pip install -U geemap 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWiAMehbUWDNxthJ8dDwqH8WTxqw5xAly6eEEPk7ZihB_o23KncOZhs

Successfully saved authorization token.


In [4]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [6]:
import os
import geemap
import leafmap

In [7]:
Map = geemap.Map()
Map.add_basemap('Hybrid')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
dataset = ee.ImageCollection('MODIS/006/MOD13Q1') \
                  .filter(ee.Filter.date('2006-02-02', '2022-02-02'))
ndvi = dataset.select('NDVI')
ndviVis = {
  'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
Map.setCenter(-6.746, -46.529, 2)
Map.addLayer(ndvi, ndviVis, 'NDVI')

In [9]:
collection = ee.ImageCollection('MODIS/006/MOD13Q1')
print(collection.size().getInfo())

513


In [10]:
median = collection.median()

In [11]:
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
                .addBands(thermalBands, None, True)

In [12]:
dataset = apply_scale_factors(median)

In [13]:
vis_natural = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

vis_nir = {
  'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
  'min': 0.0,
  'max': 0.3,
}

In [14]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'Color infrared (543)')
Map

EEException: Image.select: Pattern 'ST_B.*' did not match any bands.

In [15]:
Map = geemap.Map(center =(-2.163106, -55.126648), zoom=10)
Map.basemap()
Map

Map(center=[-2.163106, -55.126648], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [16]:
Map.setCenter(lon=-118.243683, lat=34.052235, zoom=10)
Map.basemap_demo()
Map

Map(bottom=132948.0, center=[34.052235, -118.243683], controls=(WidgetControl(options=['position', 'transparen…

In [17]:
Map = geemap.Map()   
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [18]:
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# The collection contains images for multiple years and regions in the USA.
print('Products:', dataset.aggregate_array('system:index'))

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Each product has multiple bands for describing aspects of land cover.
print('Bands:', nlcd2019.bandNames())

# Select the land cover band.
landcover = nlcd2019.select('landcover')

# Display land cover on the map.
Map.setCenter(-95, 38, 5)
Map.addLayer(landcover, {}, 'Landcover')

Products: ee.List({
  "functionInvocationValue": {
    "functionName": "AggregateFeatureCollection.array",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "USGS/NLCD_RELEASES/2019_REL/NLCD"
            }
          }
        }
      },
      "property": {
        "constantValue": "system:index"
      }
    }
  }
})
Bands: ee.List({
  "functionInvocationValue": {
    "functionName": "Image.bandNames",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.load

In [19]:
Map.search_locations

In [20]:
Map.search_loc_geom

In [21]:
!pip install -U leafmap 

Exploring Census Data


In [22]:
import leafmap

In [23]:
census_data_dict = leafmap.get_census_dict()

In [24]:
for key in census_data_dict:
    print(key)

Current
ACS 2021
ACS 2019
ACS 2018
ACS 2017
ACS 2016
ACS 2015
ACS 2014
ACS 2013
ACS 2012
ECON 2012
Census 2020
Census 2010
Physical Features
Decennial Census 2020
Decennial Census 2010
Decennial Census 2000
Decennial Physical Features


In [37]:
census_data_dict['ACS 2021']['layers']

['Alaska_Native_Regional_Corporations10465',
 'Alaska_Native_Regional_Corporations_Labels60868',
 'Alaska_Native_Village_Statistical_Areas34178',
 'Alaska_Native_Village_Statistical_Areas_Labels14319',
 'American_Indian_Joint-Use_Areas11842',
 'American_Indian_Joint-Use_Areas_Labels32943',
 'Census_Block_Groups59805',
 'Census_Block_Groups_Labels32376',
 'Census_Designated_Places65166',
 'Census_Designated_Places_Labels61459',
 'Census_Divisions8578',
 'Census_Divisions_Labels29679',
 'Census_Regions16659',
 'Census_Regions_Labels31526',
 'Census_Tracts7651',
 'Census_Tracts_Labels33558',
 'Combined_New_England_City_and_Town_Areas35117',
 'Combined_New_England_City_and_Town_Areas_Labels20328',
 'Combined_Statistical_Areas61231',
 'Combined_Statistical_Areas_Labels52090',
 'Consolidated_Cities65047',
 'Consolidated_Cities_Labels30018',
 'Counties',
 'Counties_Labels19364',
 'County_Subdivisions63689',
 'County_Subdivisions_Labels25452',
 'Elementary_School_Districts36200',
 'Elementary_

In [48]:
m = leafmap.Map()
m.add_census_data(wms="Census 2020", layer='States')
m.add_census_data(wms="Census 2020", layer="States_Labels49388")
m.add_census_data(wms="Census 2020", layer ="Tribal_Census_Tracts2987")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

You also have the option of selecting the icon visible when you click the tool in the upper right corner of dashboard. You can now load Census data without coding--just uncheck the box to replace existing census layer.

Importing Earth Engine Image Collections

In [27]:
Map = geemap.Map(add_google_map = False)

In [52]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
landcover = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD').select('landcover')
Map.addLayer(landcover, vis_params ={}, name=None, shown=True)
Map.add_legend(builtin_legend='NLCD')
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [54]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
landcover = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD').select('landcover')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [55]:
Map = geemap.Map()
Map.split_map(
    left_layer='NLCD 2016 CONUS Land Cover', right_layer='NLCD 2019 CONUS Land Cover'
)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [56]:
collection = ee.ImageCollection("USGS/NLCD")
print(collection.aggregate_array('system:id').getInfo())

['USGS/NLCD/NLCD1992', 'USGS/NLCD/NLCD2001', 'USGS/NLCD/NLCD2001_AK', 'USGS/NLCD/NLCD2001_HI', 'USGS/NLCD/NLCD2001_PR', 'USGS/NLCD/NLCD2004', 'USGS/NLCD/NLCD2006', 'USGS/NLCD/NLCD2008', 'USGS/NLCD/NLCD2011', 'USGS/NLCD/NLCD2011_AK', 'USGS/NLCD/NLCD2011_HI', 'USGS/NLCD/NLCD2011_PR', 'USGS/NLCD/NLCD2013', 'USGS/NLCD/NLCD2016']


In [57]:
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
print(collection.size().getInfo())

1407935


In [59]:
Map = geemap.Map()

In [60]:
median = collection.median()

In [61]:
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [62]:
dataset = apply_scale_factors(median)

In [63]:
vis_natural = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

vis_nir = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [64]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
landcover = ee.Image('LANDSAT/LC08/C02/T1_L2').select('landcover')

In [69]:
Map.addLayer(dataset, vis_natural, 'True color (432)')
Map.addLayer(dataset, vis_nir, 'Color infrared (543)')
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

<IPython.core.display.Javascript object>

In [70]:
vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3, 'opacity': 1.0, 'gamma': 1.0}

In [71]:
vis_params = [
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B7', 'SR_B6', 'SR_B4'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B6', 'SR_B5', 'SR_B2'], 'min': 0, 'max': 0.3},
]

In [72]:
labels = [
    'Natural Color (4, 3, 2)',
    'Color Infrared (5, 4, 3)',
    'Short-Wave Infrared (7, 6 4)',
    'Agriculture (6, 5, 2)',
]

In [68]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[-3.06, -60.01],
    zoom=4,
    ee_objects=[dataset],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…